In [1]:
# !pip install langchain
# !pip install openai
!pip install PyPDF2
!pip install faiss-cpu

     |████████████████████████████████| 232 kB 1.5 MB/s eta 0:00:01
You should consider upgrading via the '/Users/ankitkumar/Coding/Generative AI/Langchain/venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 3.1 MB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/ankitkumar/Coding/Generative AI/Langchain/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

/Users/ankitkumar/Coding/Generative AI/Langchain/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "1234"

In [5]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('cv.pdf')

In [6]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [7]:
raw_text

'Ankit Kumar\nIIT Kharagpur |+91 8607989881\nankitsuthar8607@gmail.com\n\uf092github |\uf08clinkedin |Leetcode\nSUMMARY\nSoftware developer with a distinct focus on generative AI, eager to transform ideas into intelligent solutions for\ntomorrow’s challenges\nEDUCATION\nIIT KHARAGPUR 2019-present\nMECHANICAL ENGINEERING (M. TECH DUAL), CGPA: 8.3\nEXPERIENCE\nSIMPL, ONE SIGMA TECHNOLOGY Jun’ 23 - Jul’ 23 / Bangalore\nINTERN AS SOFTWARE DEVELOPER\n•Ideated and demonstrated AI powered chatbot solution and successfully developed an react native mobile app\n•Integrated OpenAI’s API and trained on custom data via llama-index and langchain handling 80 % repetitive queries\n•Deployed code on AWS Lambda and integrated with REST APIs to optimise user interactions & response times\n•Developed and implemented a robust mobile app referral system to increase user engagement & retention\nTHE SPARKS FOUNDATION - GRADUATE ROTATIONAL INTERNSHIP PROGRAM Jun’ 21 - Jul’ 21 / Remote\nINTERN AS DATA SCIENTIS

In [8]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [9]:
len(texts)

8

In [10]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

/Users/ankitkumar/Coding/Generative AI/Langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
document_search = FAISS.from_texts(texts, embeddings)

In [12]:
document_search

In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [14]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/Users/ankitkumar/Coding/Generative AI/Langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
query = "name the person mentioned here"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/Users/ankitkumar/Coding/Generative AI/Langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Ankit Kumar'

In [16]:
query = "summarise it"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The person has experience with software development, AI, and data analysis. They have worked on various projects including a portfolio website, body movement tracking application, and a car price prediction model. They have also completed an internship as a data scientist and have experience with programming languages such as Python and JavaScript.'

In [17]:
query = "In which sports activities he is involved"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' It is mentioned in the context that he was the Captain of the Athletic Team and General Secretary of Sports and Games in Radhakrishnan Hall of Residence. He also served as the captain of a district-level kabaddi team and participated in the WIN INDIA 2.0 marathon. Therefore, he is involved in athletics, kabaddi, and marathon running.'

In [ ]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/1706.03762.pdf")

!pip install unstructured

data = loader.load()

# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

!pip install chromadb

from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

query = "Explain me about Attention is all you need"
index.query(query)